In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
path = "/content/drive/MyDrive/bm/"
sys.path.append(path)

In [3]:
import numpy as np
import torch
#from keras.datasets import mnist
import matplotlib.pyplot as plt
from dbimage_torch import DBImage
plt.rcParams['image.cmap'] = 'Greys_r'

with open(path+'data/train_data.pt', 'rb') as f:
    train_X = torch.load(f)
with open(path+'data/train_labels.pt', 'rb') as f:
    train_labels = torch.load(f)
with open(path+'data/test_data.pt', 'rb') as f:
    test_X = torch.load(f)
with open(path+'data/test_labels.pt', 'rb') as f:
    test_labels = torch.load(f)


print("Done loading MNIST")
n_train = 10000
n_test = 1000

test_X = train_X[-n_test:,:,:].float()
train_X = train_X[:n_train,:,:].float()
test_labels = train_labels[-n_test:].float()
train_labels = train_labels[:n_train].float()

lrs= [0.1,0.01,0.001,0.0001, 0.00001]
decays = [0.0001,0.000001]
moments = [0.5]
print(decays,lrs,moments)

pre_name = path+"runs/MNIST/Tuning/"

total = len(lrs)*len(decays)*len(moments)
count = 1

for l in lrs:
  for d in decays:
    for m in moments:
      print(count, " / ", total)
      name = pre_name + "lr"+str(l)  + "_d" + str(d)
      model = DBImage(28,28,10,[500,500,2000],
                batch_size=20,
                learning_rate=0.01,
                momentum=0.9,
                initial_momentum=0.5,
                weight_decay=2*10**(-5),
                tune_momentum=0.5,
                tune_rate=l, 
                tune_decay=d, 
                sparsity_target=0.05,
                sparsity_penalty=0.0,
                path_writer=name)
      model.set_images(train_X, train_labels, split=0.7)
      model.load_weights(path+"models/db_")
      model.tune(5,save=False, filename=path + "models/db_pls_no")
      acc = model.validate(full=False, no_print=True, generate=False)
      count += 1



Using CPU
Done loading MNIST
[0.0001, 1e-06] [0.1, 0.01, 0.001, 0.0001, 1e-05] [0.5]
1  /  10



Validating model
2  /  10



Validating model
3  /  10



Validating model
4  /  10



Validating model
5  /  10


...


Validating model
7  /  10



Validating model
8  /  10



Validating model
9  /  10



Validating model
10  /  10



Validating model
